https://github.com/karolzak/keras-unet/blob/master/notebooks/kz-whale-tails.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys
from pathlib import Path
from PIL import Image

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
DATA_DIR = Path("./data/tiles/ls7")

MAX_X = 255
MAX_Y = 65535  # TODO: Tilegen outputs rgb image, not grayscale

BAND_DIRS = sorted(list(DATA_DIR.glob("B*")))
TRUTH_DIR = DATA_DIR / "truth"

display(BAND_DIRS)
display(list())

In [ ]:
truth_paths = list(TRUTH_DIR.glob("*.png"))

In [ ]:
def read_fname(fname):
    # div by maxy, but mul by maxx in anticipation of preprocess_transform
    truth = np.array(Image.open(TRUTH_DIR / fname)) / (MAX_Y / MAX_X)
    bands = np.asarray([np.array(Image.open(band_dir / fname)) for band_dir in BAND_DIRS])
#     bands = bands[1:4][::-1]
    mchannel = np.dstack(bands).astype('uint8')
    return mchannel, truth

def preprocess_transform(ds):
    return (ds / MAX_X).astype('float32')

def rgb_transform(ds):
#     return np.flip(ds, 3)  # Bands 2, 3, 4 -> rgb
    return np.flip(ds[:,:,:,1:4], 3)  # Bands 1, 2, 3, 4, 5, 7 -> rgb

In [ ]:
display(truth_paths[0].name)

x0, y0 = read_fname(truth_paths[0].name)

display(x0.shape)
display(y0.shape)


In [ ]:
img_lst = []
msk_lst = []

for truth_path in truth_paths:
    img, truth = read_fname(truth_path.name)
    img_lst.append(img)
    msk_lst.append(truth)

img_np = np.asarray(img_lst)
msk_np = np.asarray(msk_lst)
msk_np = msk_np.reshape(msk_np.shape[0], msk_np.shape[1], msk_np.shape[2], 1)

del img_lst
del msk_lst

display(f'{img_np.shape=}')
display(f'{msk_np.shape=}')

In [ ]:
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=rgb_transform(img_np), mask_imgs=msk_np, nm_img_to_plot=5, figsize=6)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(img_np, msk_np, test_size=0.30, random_state=0)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

input_shape = x_train[0].shape
display(f'{input_shape=}')

In [ ]:
x_val = preprocess_transform(x_val)
y_val = preprocess_transform(y_val)

x_val_tf = tf.convert_to_tensor(x_val)
y_val_tf = tf.convert_to_tensor(y_val)

In [ ]:
from keras_unet.utils import get_augmented

train_gen = get_augmented(
    x_train, y_train, batch_size=2,
    data_gen_args = dict(
#         rescale=1 / 255,
        preprocessing_function=preprocess_transform,
        rotation_range=360.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=40,
        zoom_range=0.2,
#         brightness_range=[0.7,1.4],  # only works on 1 or 3 channel images
#         channel_shift_range=20.0,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant'
    ))

# train_gen = get_augmented(
#     x_train, y_train, batch_size=2,
#     data_gen_args=dict(
#         rotation_range=0.,
#         width_shift_range=0.00,
#         height_shift_range=0.00,
#         shear_range=0,
#         zoom_range=0.0,
#         horizontal_flip=True,
#         vertical_flip=True,
#         fill_mode='constant'
#     )
# )

del x_train
del y_train

In [ ]:
sample_batch = next(train_gen)
xx, yy = sample_batch
print(xx.shape, yy.shape)
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=rgb_transform(xx), mask_imgs=yy, nm_img_to_plot=2, figsize=6)

In [ ]:
from keras_unet.models import custom_unet

model = custom_unet(
    input_shape,
    filters=40,
    use_batch_norm=True,
    dropout=0.15,  # 0.3
    dropout_change_per_layer=0.0,
    num_layers=5
)


In [ ]:
display(model.summary())

In [ ]:
model_filename = 'model_trainchpt.h5'
callback_checkpoint = ModelCheckpoint(
    model_filename, 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True,
)


In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

model.compile(
    optimizer=Adam(), 
    #optimizer=SGD(lr=0.01, momentum=0.99),
#     loss='binary_crossentropy',
    loss=jaccard_distance,
    metrics=[iou, iou_thresholded]
)


In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=620,
    epochs=500,
    
    validation_data=(x_val_tf, y_val_tf),
    callbacks=[callback_checkpoint]
)


In [ ]:
from keras_unet.utils import plot_segm_history

plot_segm_history(history)

In [ ]:
model.load_weights(model_filename)

In [ ]:
y_pred = model.predict(x_val)

In [ ]:
from keras_unet.utils import plot_imgs

display(len(x_val))
plot_imgs(org_imgs=rgb_transform(x_val), mask_imgs=y_val, pred_imgs=y_pred, nm_img_to_plot=10, figsize=4)